<a href="https://colab.research.google.com/github/sthalles/SimCLR/blob/simclr-refactor/feature_eval/mini_batch_logistic_regression_evaluator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
import torch
import sys
import numpy as np
import os
import yaml

import matplotlib.pyplot as plt
import torchvision
import torch.nn as nn

In [34]:
#!pip install gdown

In [35]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using device:", device)

Using device: cuda


In [36]:
model = torchvision.models.squeezenet1_0(pretrained=False)
additional_fc_layer = nn.Sequential(nn.Linear(1000, 512), nn.ReLU(), nn.Linear(512, 128))
model=nn.Sequential(model, additional_fc_layer).to(device)


In [37]:
model

Sequential(
  (0): SqueezeNet(
    (features): Sequential(
      (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      (3): Fire(
        (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
        (squeeze_activation): ReLU(inplace=True)
        (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
        (expand1x1_activation): ReLU(inplace=True)
        (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (expand3x3_activation): ReLU(inplace=True)
      )
      (4): Fire(
        (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
        (squeeze_activation): ReLU(inplace=True)
        (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
        (expand1x1_activation): ReLU(inplace=True)
        (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (expand3x3

In [38]:
# freeze all layers but the last fc
'''for name, param in model.named_parameters():
    if not name.startswith('1.'):
        param.requires_grad = False
        print(name)

parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
#print(parameters.name)
assert len(parameters) == 4  # fc.weight, fc.bias'''

"for name, param in model.named_parameters():\n    if not name.startswith('1.'):\n        param.requires_grad = False\n        print(name)\n\nparameters = list(filter(lambda p: p.requires_grad, model.parameters()))\n#print(parameters.name)\nassert len(parameters) == 4  # fc.weight, fc.bias"

In [39]:
'''def get_file_id_by_model(folder_name):
  file_id = {'resnet18_100-epochs_stl10': '14_nH2FkyKbt61cieQDiSbBVNP8-gtwgF',
             'resnet18_100-epochs_cifar10': '1lc2aoVtrAetGn0PnTkOyFzPCIucOJq7C',
             'resnet50_50-epochs_stl10': '1ByTKAUsdm_X7tLcii6oAEl5qFRqRMZSu'}
  return file_id.get(folder_name, "Model not found.")'''

'def get_file_id_by_model(folder_name):\n  file_id = {\'resnet18_100-epochs_stl10\': \'14_nH2FkyKbt61cieQDiSbBVNP8-gtwgF\',\n             \'resnet18_100-epochs_cifar10\': \'1lc2aoVtrAetGn0PnTkOyFzPCIucOJq7C\',\n             \'resnet50_50-epochs_stl10\': \'1ByTKAUsdm_X7tLcii6oAEl5qFRqRMZSu\'}\n  return file_id.get(folder_name, "Model not found.")'

In [40]:
'''folder_name = 'resnet50_50-epochs_stl10'
file_id = get_file_id_by_model(folder_name)
print(folder_name, file_id)'''

"folder_name = 'resnet50_50-epochs_stl10'\nfile_id = get_file_id_by_model(folder_name)\nprint(folder_name, file_id)"

In [41]:
# download and extract model files
#os.system('gdown https://drive.google.com/uc?id={}'.format(file_id))
#os.system('unzip {}'.format(folder_name))
#!ls

In [42]:
'''import zipfile
import os

zip_file = "resnet50_50-epochs_stl10.zip"

# Get the current directory where the ZIP file is located
current_directory = os.path.dirname(os.path.abspath(zip_file))

# Open the ZIP file for reading
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    # Extract all the contents into the same folder
    zip_ref.extractall(current_directory)
'''


'import zipfile\nimport os\n\nzip_file = "resnet50_50-epochs_stl10.zip"\n\n# Get the current directory where the ZIP file is located\ncurrent_directory = os.path.dirname(os.path.abspath(zip_file))\n\n# Open the ZIP file for reading\nwith zipfile.ZipFile(zip_file, \'r\') as zip_ref:\n    # Extract all the contents into the same folder\n    zip_ref.extractall(current_directory)\n'

In [43]:
!ls

checkpoint_0005.pth.tar
checkpoint_0040.pth.tar
config.yml
data
events.out.tfevents.1610927742.4cb2c837708d.2694093.0
mini_batch_logistic_regression_evaluator.ipynb
squeeze_checkpoint_0100.pth.tar
squeezenet_checkpoint_0001.pth.tar
squeezenet_checkpoint_0100.pth.tar
training.log


In [44]:
'''import os

zip_file = "resnet50_50-epochs_stl10.zip"

# Get the size of the ZIP file in bytes
file_size_bytes = os.path.getsize(zip_file)

# Convert bytes to a human-readable format (e.g., kilobytes, megabytes, etc.)
def convert_size(size_bytes):
    # Size conversion
    for unit in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if size_bytes < 1024.0:
            return f"{size_bytes:.2f} {unit}"
        size_bytes /= 1024.0

# Get human-readable file size
file_size_human_readable = convert_size(file_size_bytes)

print(f"The size of '{zip_file}' is: {file_size_human_readable}")
'''

'import os\n\nzip_file = "resnet50_50-epochs_stl10.zip"\n\n# Get the size of the ZIP file in bytes\nfile_size_bytes = os.path.getsize(zip_file)\n\n# Convert bytes to a human-readable format (e.g., kilobytes, megabytes, etc.)\ndef convert_size(size_bytes):\n    # Size conversion\n    for unit in [\'bytes\', \'KB\', \'MB\', \'GB\', \'TB\']:\n        if size_bytes < 1024.0:\n            return f"{size_bytes:.2f} {unit}"\n        size_bytes /= 1024.0\n\n# Get human-readable file size\nfile_size_human_readable = convert_size(file_size_bytes)\n\nprint(f"The size of \'{zip_file}\' is: {file_size_human_readable}")\n'

In [45]:
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import datasets

In [46]:
def get_stl10_data_loaders(download, shuffle=False, batch_size=256):
  train_dataset = datasets.STL10('./data', split='train', download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=0, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.STL10('./data', split='test', download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=10, drop_last=False, shuffle=shuffle)
  return train_loader, test_loader

def get_cifar10_data_loaders(download, shuffle=False, batch_size=256):
  train_dataset = datasets.CIFAR10('./data', train=True, download=download,
                                  transform=transforms.ToTensor())

  train_loader = DataLoader(train_dataset, batch_size=batch_size,
                            num_workers=0, drop_last=False, shuffle=shuffle)
  
  test_dataset = datasets.CIFAR10('./data', train=False, download=download,
                                  transform=transforms.ToTensor())

  test_loader = DataLoader(test_dataset, batch_size=2*batch_size,
                            num_workers=10, drop_last=False, shuffle=shuffle)
  return train_loader, test_loader

In [47]:
with open(os.path.join('./config.yml')) as file:
  config = yaml.load(file, Loader=yaml.FullLoader)

In [48]:
'''if config['arch'] == 'resnet18':
  model = torchvision.models.resnet18(pretrained=False, num_classes=10).to(device)
elif config['arch'] == 'resnet50':
  model = torchvision.models.resnet50(pretrained=False, num_classes=10).to(device)
elif config['arch'] == 'squeezenet':
  print("squeezenet")
  model = torchvision.models.squeezenet1_0(pretrained=False).to(device).to(device)
  # or use squeezenet1_1 depending on the version you want'''

'if config[\'arch\'] == \'resnet18\':\n  model = torchvision.models.resnet18(pretrained=False, num_classes=10).to(device)\nelif config[\'arch\'] == \'resnet50\':\n  model = torchvision.models.resnet50(pretrained=False, num_classes=10).to(device)\nelif config[\'arch\'] == \'squeezenet\':\n  print("squeezenet")\n  model = torchvision.models.squeezenet1_0(pretrained=False).to(device).to(device)\n  # or use squeezenet1_1 depending on the version you want'

In [49]:
'''rmodel=model = torchvision.models.resnet18(pretrained=False).to(device)
rmodel'''

'rmodel=model = torchvision.models.resnet18(pretrained=False).to(device)\nrmodel'

In [50]:
checkpoint = torch.load('squeezenet_checkpoint_0100.pth.tar', map_location=device)
state_dict = checkpoint['state_dict']
print(state_dict.keys())

odict_keys(['backbone.0.features.0.weight', 'backbone.0.features.0.bias', 'backbone.0.features.3.squeeze.weight', 'backbone.0.features.3.squeeze.bias', 'backbone.0.features.3.expand1x1.weight', 'backbone.0.features.3.expand1x1.bias', 'backbone.0.features.3.expand3x3.weight', 'backbone.0.features.3.expand3x3.bias', 'backbone.0.features.4.squeeze.weight', 'backbone.0.features.4.squeeze.bias', 'backbone.0.features.4.expand1x1.weight', 'backbone.0.features.4.expand1x1.bias', 'backbone.0.features.4.expand3x3.weight', 'backbone.0.features.4.expand3x3.bias', 'backbone.0.features.5.squeeze.weight', 'backbone.0.features.5.squeeze.bias', 'backbone.0.features.5.expand1x1.weight', 'backbone.0.features.5.expand1x1.bias', 'backbone.0.features.5.expand3x3.weight', 'backbone.0.features.5.expand3x3.bias', 'backbone.0.features.7.squeeze.weight', 'backbone.0.features.7.squeeze.bias', 'backbone.0.features.7.expand1x1.weight', 'backbone.0.features.7.expand1x1.bias', 'backbone.0.features.7.expand3x3.weight'

In [51]:
print(len(state_dict.keys()))

56


In [52]:
#checkpoint = torch.load('checkpoint_0040.pth.tar', map_location=device)
checkpoint = torch.load('squeezenet_checkpoint_0001.pth.tar', map_location=device)
state_dict = checkpoint['state_dict']

for k in list(state_dict.keys()):

  if k.startswith('backbone.'):
    #print("yes")
    if k.startswith('backbone') and not k.startswith('backbone.fc'):
      print(k)
      # remove prefix
      state_dict[k[len("backbone."):]] = state_dict[k]

  del state_dict[k]

backbone.0.features.0.weight
backbone.0.features.0.bias
backbone.0.features.3.squeeze.weight
backbone.0.features.3.squeeze.bias
backbone.0.features.3.expand1x1.weight
backbone.0.features.3.expand1x1.bias
backbone.0.features.3.expand3x3.weight
backbone.0.features.3.expand3x3.bias
backbone.0.features.4.squeeze.weight
backbone.0.features.4.squeeze.bias
backbone.0.features.4.expand1x1.weight
backbone.0.features.4.expand1x1.bias
backbone.0.features.4.expand3x3.weight
backbone.0.features.4.expand3x3.bias
backbone.0.features.5.squeeze.weight
backbone.0.features.5.squeeze.bias
backbone.0.features.5.expand1x1.weight
backbone.0.features.5.expand1x1.bias
backbone.0.features.5.expand3x3.weight
backbone.0.features.5.expand3x3.bias
backbone.0.features.7.squeeze.weight
backbone.0.features.7.squeeze.bias
backbone.0.features.7.expand1x1.weight
backbone.0.features.7.expand1x1.bias
backbone.0.features.7.expand3x3.weight
backbone.0.features.7.expand3x3.bias
backbone.0.features.8.squeeze.weight
backbone.0.

In [53]:
print(len(state_dict.keys()))

56


In [54]:
log = model.load_state_dict(state_dict, strict=False)
#assert log.missing_keys == ['fc.weight', 'fc.bias']

In [55]:
log

<All keys matched successfully>

In [56]:
#additional_fc_layer = torch.nn.Linear(1000, 10)

# Access the existing classifier and add the additional FC layer
#model.classifier.add_module('fc', additional_fc_layer)

In [57]:
if config['dataset_name'] == 'cifar10':
  train_loader, test_loader = get_cifar10_data_loaders(download=True)
elif config['dataset_name'] == 'stl10':
  train_loader, test_loader = get_stl10_data_loaders(download=True)
print("Dataset:", config['dataset_name'])

Files already downloaded and verified
Files already downloaded and verified
Dataset: stl10


In [58]:
# freeze all layers but the last fc
'''for name, param in model.named_parameters():
    if name not in ['fc.weight', 'fc.bias']:
        param.requires_grad = False

parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
print(parameters)
assert len(parameters) == 2  # fc.weight, fc.bias'''

"for name, param in model.named_parameters():\n    if name not in ['fc.weight', 'fc.bias']:\n        param.requires_grad = False\n\nparameters = list(filter(lambda p: p.requires_grad, model.parameters()))\nprint(parameters)\nassert len(parameters) == 2  # fc.weight, fc.bias"

In [61]:
# freeze all layers but the last fc
for name, param in model.named_parameters():
    if not name.startswith('1.2'):
        param.requires_grad = False
        print(name)

parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
#print(parameters.name)
assert len(parameters) == 2  # fc.weight, fc.bias

0.features.0.weight
0.features.0.bias
0.features.3.squeeze.weight
0.features.3.squeeze.bias
0.features.3.expand1x1.weight
0.features.3.expand1x1.bias
0.features.3.expand3x3.weight
0.features.3.expand3x3.bias
0.features.4.squeeze.weight
0.features.4.squeeze.bias
0.features.4.expand1x1.weight
0.features.4.expand1x1.bias
0.features.4.expand3x3.weight
0.features.4.expand3x3.bias
0.features.5.squeeze.weight
0.features.5.squeeze.bias
0.features.5.expand1x1.weight
0.features.5.expand1x1.bias
0.features.5.expand3x3.weight
0.features.5.expand3x3.bias
0.features.7.squeeze.weight
0.features.7.squeeze.bias
0.features.7.expand1x1.weight
0.features.7.expand1x1.bias
0.features.7.expand3x3.weight
0.features.7.expand3x3.bias
0.features.8.squeeze.weight
0.features.8.squeeze.bias
0.features.8.expand1x1.weight
0.features.8.expand1x1.bias
0.features.8.expand3x3.weight
0.features.8.expand3x3.bias
0.features.9.squeeze.weight
0.features.9.squeeze.bias
0.features.9.expand1x1.weight
0.features.9.expand1x1.bias


In [62]:
parameters[0].shape;
parameters[1].shape

torch.Size([128])

In [63]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=0.0008)
criterion = torch.nn.CrossEntropyLoss().to(device)

In [64]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [65]:
epochs = 100
for epoch in range(epochs):
  top1_train_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(train_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
    loss = criterion(logits, y_batch)
    top1 = accuracy(logits, y_batch, topk=(1,))
    top1_train_accuracy += top1[0]

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  top1_train_accuracy /= (counter + 1)
  top1_accuracy = 0
  top5_accuracy = 0
  for counter, (x_batch, y_batch) in enumerate(test_loader):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    logits = model(x_batch)
  
    top1, top5 = accuracy(logits, y_batch, topk=(1,5))
    top1_accuracy += top1[0]
    top5_accuracy += top5[0]
  
  top1_accuracy /= (counter + 1)
  top5_accuracy /= (counter + 1)
  print(f"Epoch {epoch}\tTop1 Train accuracy {top1_train_accuracy.item()}\tTop1 Test accuracy: {top1_accuracy.item()}\tTop5 test acc: {top5_accuracy.item()}")

Epoch 0	Top1 Train accuracy 8.990118980407715	Top1 Test accuracy: 10.07080078125	Top5 test acc: 50.1708984375
Epoch 1	Top1 Train accuracy 10.042510032653809	Top1 Test accuracy: 10.51025390625	Top5 test acc: 54.9755859375
Epoch 2	Top1 Train accuracy 14.1475191116333	Top1 Test accuracy: 19.47265625	Top5 test acc: 65.31494140625
Epoch 3	Top1 Train accuracy 20.54917335510254	Top1 Test accuracy: 21.49658203125	Top5 test acc: 74.40673828125
Epoch 4	Top1 Train accuracy 22.03929328918457	Top1 Test accuracy: 22.67578125	Top5 test acc: 79.31640625
Epoch 5	Top1 Train accuracy 23.59145164489746	Top1 Test accuracy: 24.47509765625	Top5 test acc: 80.791015625
Epoch 6	Top1 Train accuracy 24.9609375	Top1 Test accuracy: 25.8154296875	Top5 test acc: 82.080078125
Epoch 7	Top1 Train accuracy 26.965763092041016	Top1 Test accuracy: 27.177734375	Top5 test acc: 83.22021484375
Epoch 8	Top1 Train accuracy 28.215763092041016	Top1 Test accuracy: 28.48876953125	Top5 test acc: 84.53125
Epoch 9	Top1 Train accuracy 29